### Install azureml library

In [2]:
! pip install azureml-core

     |████████████████████████████████| 2.2 MB 4.2 MB/s 
     |████████████████████████████████| 101 kB 10.9 MB/s 
     |████████████████████████████████| 40 kB 5.3 MB/s 
     |████████████████████████████████| 1.3 MB 37.7 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 312 kB 46.2 MB/s 
     |████████████████████████████████| 147 kB 60.8 MB/s 
     |████████████████████████████████| 3.0 MB 38.4 MB/s 
     |████████████████████████████████| 94 kB 3.0 MB/s 
     |████████████████████████████████| 547 kB 62.7 MB/s 
     |████████████████████████████████| 55 kB 3.5 MB/s 
     |████████████████████████████████| 796 kB 42.0 MB/s 
     |████████████████████████████████| 141 kB 47.7 MB/s 
     |████████████████████████████████| 85 kB 4.2 MB/s 
     |████████████████████████████████| 165 kB 56.8 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
     |████████████████████████████████| 45 kB 3.4 MB/s 
     |███████████████████████

### Create workspace from Azure Machine Learning workspace

In [4]:
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

source_dir = "./source_dir/"

ws = Workspace.from_config(path = source_dir + "config.json")
print(ws)

Workspace.create(name='Test-Deployment-Space', subscription_id='9d7c60f0-d3e1-49c3-b70b-080e861838c1', resource_group='Test-Deployment')


### Import libraries

In [5]:
import os
import json

import tensorflow as tf

### Load tokenizer from saved tokenizer

In [6]:
with open(os.path.join('./outputs/model1/assets/tokenizer.json')) as f:
    data = json.load(f)
    tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(data)

### Load the maximum sequence length from all the Commonlit provided excerpts stored in txt file

In [7]:
with open('./outputs/model1/assets/max_length.txt', 'r') as text:
    max_length = int(text.read())

### Test loading of pre-trained model

In [8]:
model_load = tf.keras.models.load_model('./outputs/model1')
model_load.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 215, 90)           2581650   
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               224256    
_________________________________________________________________
dense (Dense)                (None, 1)                 257       
Total params: 2,806,163
Trainable params: 2,806,163
Non-trainable params: 0
_________________________________________________________________


### Register saved model on Azure

In [19]:
from azureml.core.model import Model

# Register model
model = Model.register(workspace = ws, 
                       model_name = "Commonlit-BiLSTM",
                       model_path = "./outputs/model1",
                       model_framework = "TensorFlow")

Registering model Commonlit-BiLSTM


### Create environment with necessary dependencies

In [10]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig

env = Environment(name = "Commonlit-BiLSTM")
conda_dep = CondaDependencies()
conda_dep.add_conda_package("numpy")
conda_dep.add_pip_package("keras")

conda_dep.add_pip_package('tensorflow==2.6.0')

conda_dep.add_pip_package("azureml-defaults")
conda_dep.add_pip_package("azureml")
conda_dep.add_pip_package("azureml-contrib-functions")

env.python.conda_dependencies = conda_dep

### Create inference configuration using scoring function

In [12]:
inference_config = InferenceConfig(
    environment = env,
    source_directory = "./source_dir",
    entry_script = "./echo_score_LOAD.py",
)

### Create Azure Container Instance and deploy model to container

In [14]:
from azureml.core.webservice import AciWebservice
aci_config = AciWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 2)
service = Model.deploy(
    ws,
    "commonlit-bi-lstm",
    [model],
    inference_config,
    aci_config,
    overwrite = True,
)
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-08-27 23:29:15+00:00 Creating Container Registry if not exists.
2021-08-27 23:29:16+00:00 Registering the environment.
2021-08-27 23:29:20+00:00 Use the existing image.
2021-08-27 23:29:20+00:00 Generating deployment configuration.
2021-08-27 23:29:21+00:00 Submitting deployment to compute..
2021-08-27 23:29:28+00:00 Checking the status of deployment commonlit-bi-lstm..
2021-08-27 23:49:53+00:00 Checking the status of inference endpoint commonlit-bi-lstm.
Succeeded
ACI service creation operation finished, operation "Succeeded"


### Endpoint Consumption - Test 1

In [16]:
import requests
import json

uri = service.scoring_uri
requests.get("http://8533275d-6c1e-4a49-9b7b-d7b4f9efaf0e.australiaeast.azurecontainer.io/score")
headers = {"Content-Type": "application/json"}
data = {
    'data': ['Cell division is the process by which a parent cell divides into two or more daughter cells. Cell division usually occurs as part of a larger cell cycle.\n In eukaryotes, there are two distinct types of cell division: a vegetative division, whereby each daughter cell is genetically identical to the parent cell (mitosis), and a reproductive cell division, whereby the number of chromosomes in the daughter cells is reduced by half, to produce haploid gametes (meiosis). \nMeiosis results in four haploid daughter cells by undergoing one round of DNA replication followed by two divisions: homologous chromosomes are separated in the first division, and sister chromatids are separated in the second division.\nBoth of these cell division cycles are used in sexually reproducing organisms at some point in their life cycle, and both are believed to be present in the last eukaryotic common ancestor. Prokaryotes also undergo a vegetative cell division known as binary fission, where their genetic material is segregated equally into two daughter cells. All cell divisions, regardless of organism, are preceded by a single round of DNA replication.']
}
data = json.dumps(data)
response = requests.post(uri, data = data, headers = headers)
print(response.json())

[-2.2895405292510986]


### Endpoint Consumption - Test 2

In [17]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
data = {
    'data': ['Dotty continued to go to Mrs. Gray\'s every night with the milk. Sometimes Katie went with her, and then they always paused a while under the acorn-tree and played "King and Queen." Dotty said she wished they could ever remember to bring their nipperkins, for in that case the milk would taste a great deal more like nectar. The "nipperkins" were a pair of handled cups which the children supposed to be silver, and which they always used at table.\nDotty knew she was doing wrong every time she played "King and Queen." She knew the milk was not hers, but Mrs. Gray\'s; still she said to herself, "Ruthie needn\'t give so much measure, all pressed down and run over. If Queenie and I should drink a great deal more, there would always be a quart left. Yes, I know there would."\nMrs. Gray never said anything about the milk; she merely poured it out in a pan, and gave back the pail to Dotty, asking her at the same time as many questions as the child would stay to hear.',
             'Cell division is the process by which a parent cell divides into two or more daughter cells. Cell division usually occurs as part of a larger cell cycle.\n In eukaryotes, there are two distinct types of cell division: a vegetative division, whereby each daughter cell is genetically identical to the parent cell (mitosis), and a reproductive cell division, whereby the number of chromosomes in the daughter cells is reduced by half, to produce haploid gametes (meiosis). \nMeiosis results in four haploid daughter cells by undergoing one round of DNA replication followed by two divisions: homologous chromosomes are separated in the first division, and sister chromatids are separated in the second division.\nBoth of these cell division cycles are used in sexually reproducing organisms at some point in their life cycle, and both are believed to be present in the last eukaryotic common ancestor. Prokaryotes also undergo a vegetative cell division known as binary fission, where their genetic material is segregated equally into two daughter cells. All cell divisions, regardless of organism, are preceded by a single round of DNA replication.'],
}

body = str.encode(json.dumps(data))

url = 'http://8533275d-6c1e-4a49-9b7b-d7b4f9efaf0e.australiaeast.azurecontainer.io/score'
api_key = '' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))


b'[-0.22421894967556, -2.2895405292510986]'
